In [1]:
import os, sys, glob, math, csv

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [22]:

##########################################################################
##          movie files and parameters
##########################################################################
DATAHOME = '/home/staff1/ctorney/data/wildebeest_front/'
inputname = DATAHOME + '/wildfront.csv'
dfMovies = pd.read_csv(inputname)


dfMovies = dfMovies[dfMovies['ir']==0]
beestList = np.zeros(shape=(0,3))
##########################################################################
##          loop through movies in list
##########################################################################
for index, d in dfMovies.iterrows():
    filename = DATAHOME + d['filename']
    direct, ext = os.path.split(filename)
    noext, _ = os.path.splitext(ext)
    inputdatafile = direct + '/proc/' +  noext + '_POS.txt'
    warpsfile = direct + '/proc/' + noext + '_WARP.npy'
    data = np.genfromtxt(inputdatafile,delimiter=',') #,dtype=None, names=True)
    
    
    
    timepoints=data[:,0]
    w_ids=(data[:,1])
   
    for i in range(int(np.max(timepoints)//(4*60))):
        indexes = (timepoints>(i*4*60))&(timepoints<((i+1)*4*60))
        t_w_id = w_ids[indexes]
        t_tp = timepoints[indexes]
        for w in np.unique(t_w_id):
            length = np.max(t_tp[t_w_id==w])-np.min(t_tp[t_w_id==w])
            if length>(59*4):
                beestList=np.append(beestList,np.array([[index,i,w]]),axis=0)
                count+=1
        
    
beest2000i = np.random.choice(beestList.shape[0], 2000, replace=False)
beest2000i = np.sort(beest2000i)
beest2000 = beestList[beest2000i,:]
np.save('beest2000.npy',beest2000)

In [41]:
beest1000=np.load('beest2000.npy')
##########################################################################
##          movie files and parameters
##########################################################################
DATAHOME = '/home/staff1/ctorney/data/wildebeest_front/'
inputname = DATAHOME + '/wildfront.csv'
dfMovies = pd.read_csv(inputname)


dfMovies = dfMovies[dfMovies['ir']==0]
bcount=0
##########################################################################
##          loop through movies in list
##########################################################################
BFA = np.zeros((0,11))
for index, d in dfMovies.iterrows():
    
    filename = DATAHOME + d['filename']
    direct, ext = os.path.split(filename)
    noext, _ = os.path.splitext(ext)
    inputdatafile = direct + '/proc/' +  noext + '_POS.txt'
    #warpsfile = direct + '/proc/' + noext + '_WARP.npy'
    BLfile = direct + '/proc/' +  noext + '_BL.npy'
    bodylengths = np.load(BLfile)
    data = np.genfromtxt(inputdatafile,delimiter=',') #,dtype=None, names=True)
    
    thisFile = beest1000[beest1000[:,0]==index]
    count=0
    timepoints=data[:,0]
    w_ids=(data[:,1])
    xpos = 0.5*(data[:,2]+data[:,4])
    ypos = 0.5*(data[:,3]+data[:,5])
    movieArray = np.zeros((0,11))
    for i in range(int(np.max(timepoints)//(4*60))):
        indexes = (timepoints>(i*4*60))&(timepoints<((i+1)*4*60))
        t_w_id = w_ids[indexes]
        t_tp = timepoints[indexes]
        t_xp = xpos[indexes]
        t_yp = ypos[indexes]
        beestList = beest1000[(beest1000[:,0]==index)&(beest1000[:,1]==i),2]
        for b in beestList:
            if bcount==1000: 
                break
            beestTime = t_tp[t_w_id==b]
            beestX = t_xp[t_w_id==b]
            beestY = t_yp[t_w_id==b]
            minT = int(np.min(beestTime))
            maxT = int(np.max(beestTime))
            if np.sum(bodylengths[minT:maxT]==0)>0:
                continue
            ibeestX = np.interp(np.arange(minT,maxT+1),beestTime,beestX)
            ibeestY = np.interp(np.arange(minT,maxT+1),beestTime,beestY)
            ts = np.arange(minT,maxT+1)
            beestArray = np.zeros((0,11))
            for ft in range(len(ts)):
                bl = bodylengths[ts[ft]]
                cx = ibeestX[ft]/bl
                cy = ibeestY[ft]/bl
                otherX = t_xp[(t_w_id!=b)&(t_tp==ts[ft])]/bl
                otherY = t_yp[(t_w_id!=b)&(t_tp==ts[ft])]/bl
                diffs = (cx-otherX)**2+(cy-otherY)**2
                if len(diffs)<5:
                    beestArray = np.zeros((0,11))
                    break
                k = 4

                idx = np.argpartition(diffs, k)
                beestArray = np.vstack((beestArray, np.hstack((bcount, cx,cy,otherX[idx[:k]].T,otherY[idx[:k]].T))))
            if len(beestArray)>0:
                movieArray = np.vstack((movieArray, beestArray))
                bcount+=1
        
    BFA = np.vstack((BFA, movieArray))
    

In [46]:
np.save('BFA.npy',BFA)


In [38]:
beestArray = np.zeros((0,11))

In [45]:
np.unique(BFA[:,0]).shape

(1000,)

In [33]:
k = min(4,len(diffs)-1)

In [37]:
k

3

In [45]:
beestList=np.append(beestList,np.array([[index,w]]),axis=0)

In [75]:
beestList.shape

(301, 3)

In [71]:
np.max(timepoints)


2239.0

In [73]:
for i in range(int(np.max(timepoints)//(4*60))):
    print((i+1)*4*60)

240
480
720
960
1200
1440
1680
1920
2160
